In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [5]:
from src.inference import get_feature_store

In [ ]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*28)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

In [19]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 01:00:00,2,0
1,2025-02-04 02:00:00,2,0
2,2025-02-04 03:00:00,2,0
3,2025-02-04 04:00:00,2,0
4,2025-02-04 05:00:00,2,0
...,...,...,...
167161,2025-03-03 14:00:00,263,119
167162,2025-03-03 15:00:00,263,119
167163,2025-03-03 16:00:00,263,89
167164,2025-03-03 17:00:00,263,114


In [20]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167166 entries, 0 to 169424
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         167166 non-null  datetime64[us]
 1   pickup_location_id  167166 non-null  int32         
 2   rides               167166 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.8 MB


In [21]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [22]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167166 entries, 0 to 169424
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         167166 non-null  datetime64[us]
 1   pickup_location_id  167166 non-null  int32         
 2   rides               167166 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.8 MB


In [26]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*27, step_size=23)

In [27]:
features

,rides_t-648,rides_t-647,rides_t-646,rides_t-645,rides_t-644,rides_t-643,rides_t-642,rides_t-641,rides_t-640,rides_t-639,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,1,0,0,1,1,2,1,2,0,...,0,1,0,0,0,0,0,0,196,2025-02-25 22:00:00
1,32,108,136,24,123,145,229,57,130,85,...,108,55,108,132,39,56,6,109,263,2025-02-28 07:00:00
2,2,0,1,3,0,0,24,0,0,0,...,1,1,25,0,0,0,0,16,255,2025-03-01 22:00:00
3,0,0,0,1,0,1,0,0,0,1,...,0,1,0,1,0,0,0,1,210,2025-02-23 15:00:00
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,53,2025-02-28 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,1,0,1,0,0,1,0,0,0,...,1,0,0,1,0,2,0,1,122,2025-02-06 02:00:00
247,47,173,243,3,177,148,423,199,173,156,...,136,37,18,2,216,138,148,198,142,2025-02-04 22:00:00
248,67,70,69,81,75,122,165,22,93,98,...,54,0,32,121,56,121,110,19,231,2025-02-19 20:00:00
249,0,0,0,0,0,1,0,0,0,1,...,1,0,0,1,0,2,0,0,205,2025-02-16 06:00:00


In [28]:
from src.inference import load_batch_of_features_from_store

In [29]:
current_date

Timestamp('2025-03-04 00:33:46.956046+0000', tz='Etc/UTC')

In [31]:
features=load_batch_of_features_from_store(current_date)
features

2025-03-03 19:36:52,300 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:36:52,301 INFO: Initializing external client
2025-03-03 19:36:52,302 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:36:53,342 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652
Fetching data from 2025-02-03 00:33:46.956046+00:00 to 2025-03-03 23:33:46.956046+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.57s) 


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 16:00:00
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,3,2025-03-03 16:00:00
2,1,4,0,1,3,3,6,2,3,6,...,11,4,3,5,1,2,2,3,4,2025-03-03 16:00:00
3,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,6,2025-03-03 16:00:00
4,1,1,1,5,1,2,0,0,0,0,...,5,3,3,2,8,3,4,1,7,2025-03-03 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,1,0,0,0,259,2025-03-03 16:00:00
247,3,1,1,3,0,1,0,0,0,0,...,3,0,0,1,4,3,2,5,260,2025-03-03 16:00:00
248,38,30,41,24,17,16,11,3,1,3,...,9,12,7,16,25,36,32,26,261,2025-03-03 16:00:00
249,81,98,73,43,34,22,4,6,1,3,...,172,124,118,98,79,71,82,97,262,2025-03-03 16:00:00


In [32]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

2025-03-03 19:37:07,048 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:37:07,049 INFO: Initializing external client
2025-03-03 19:37:07,050 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:37:07,743 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652

In [33]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [34]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,6,0.0
4,7,2.0
...,...,...
246,259,0.0
247,260,2.0
248,261,38.0
249,262,84.0


In [35]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 237, 236, 161, 162, 138, 163, 140, 239, 142], dtype=int32)